In [89]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso,Ridge,ElasticNet, LinearRegression
import statsmodels.api as sm

In [49]:
data = pd.read_csv(r"C:\Users\aykut\OneDrive\Belgeler\GitHub\IH_RH_DA_FT_AUG_2022\Class_Materials\Case_Studies\Customer_Analysis_Case_Study\Data\Data_Marketing_Customer_Analysis_Round3.csv")
data.head()

,region,customer_lifetime_value,response,coverage,education,effective_to_date,month,employment_status,gender,income,...,months_since_policy_inception,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size
0,central,4809,no,basic,college,2/18/11,feb,employed,m,48029,...,52,0,9,corporate auto,corporate l3,offer3,agent,292,four-door car,medsize
1,west region,2228,no,basic,college,1/18/11,jan,unemployed,f,92260,...,26,0,1,personal auto,personal l3,offer4,call center,744,four-door car,medsize
2,east,14947,no,basic,bachelor,2/10/11,feb,employed,m,22139,...,31,0,2,personal auto,personal l3,offer3,call center,480,suv,medsize
3,north west,22332,yes,extended,college,1/11/11,jan,employed,m,49078,...,3,0,2,corporate auto,corporate l3,offer2,branch,484,four-door car,medsize
4,north west,9025,no,premium,bachelor,1/17/11,jan,medical leave,f,23675,...,31,0,7,personal auto,personal l2,offer1,branch,707,four-door car,medsize


In [50]:
numerical = data.select_dtypes(include=np.number)
categorical = data.select_dtypes(include=object)

## Data splitting

In [7]:
y= data['total_claim_amount']

In [12]:
X = data.drop(['total_claim_amount'],axis =1)

In [13]:
X

,region,customer_lifetime_value,response,coverage,education,effective_to_date,month,employment_status,gender,income,...,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size
0,central,4809,no,basic,college,2/18/11,feb,employed,m,48029,...,7,52,0,9,corporate auto,corporate l3,offer3,agent,four-door car,medsize
1,west region,2228,no,basic,college,1/18/11,jan,unemployed,f,92260,...,3,26,0,1,personal auto,personal l3,offer4,call center,four-door car,medsize
2,east,14947,no,basic,bachelor,2/10/11,feb,employed,m,22139,...,34,31,0,2,personal auto,personal l3,offer3,call center,suv,medsize
3,north west,22332,yes,extended,college,1/11/11,jan,employed,m,49078,...,10,3,0,2,corporate auto,corporate l3,offer2,branch,four-door car,medsize
4,north west,9025,no,premium,bachelor,1/17/11,jan,medical leave,f,23675,...,33,31,0,7,personal auto,personal l2,offer1,branch,four-door car,medsize
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10684,central,15563,no,premium,bachelor,1/19/11,jan,unemployed,f,61541,...,12,40,0,7,personal auto,personal l1,offer3,web,luxury car,medsize
10685,north west,5259,no,basic,college,1/6/11,jan,employed,f,61146,...,7,68,0,6,personal auto,personal l3,offer2,branch,four-door car,medsize
10686,central,23893,no,extended,bachelor,2/6/11,feb,employed,f,39837,...,11,63,0,2,corporate auto,corporate l3,offer1,web,luxury suv,medsize
10687,west region,11971,no,premium,college,2/13/11,feb,employed,f,64195,...,0,27,4,6,personal auto,personal l1,offer1,branch,suv,medsize


In [51]:
categorical['number_of_open_complaints'] = data['number_of_open_complaints']

In [19]:
education_level = {'high school or below':1,  #converting education column to ordinal values
                  'bachelor':2,
                   'college':3,
                  'bachelor':4,
                   'master':5,
                   'doctor':6}
education = []
for x in categorical['education']:
    if x in education_level:
        education.append(education_level[x])

In [52]:
categorical['education'] = education

In [53]:
categorical['coverage'] = [1 if x == 'basic' else               #conversion of coverage column to ordinal values
                           2 if  x =='extended' else 
                           3 
                           for x in categorical['coverage']]


In [54]:
categorical.drop('effective_to_date', axis=1, inplace=True)

In [23]:
categorical['policy'] = ['personal' if x in ('personal l1','personal l2','personal l3') else 
                         'corporate' if x in ('corporate l3','corporate l2', 'corporate l1') else 
                         'special'  for x in categorical['policy']]

In [55]:
categorical_features=pd.get_dummies(categorical, drop_first=True)

In [56]:
categorical_features

,coverage,education,number_of_open_complaints,region_east,region_north west,region_west region,response_yes,month_jan,employment_status_employed,employment_status_medical leave,...,sales_channel_branch,sales_channel_call center,sales_channel_web,vehicle_class_luxury car,vehicle_class_luxury suv,vehicle_class_sports car,vehicle_class_suv,vehicle_class_two-door car,vehicle_size_medsize,vehicle_size_small
0,1,3,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
1,1,3,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
2,1,4,0,1,0,0,0,0,1,0,...,0,1,0,0,0,0,1,0,1,0
3,2,3,0,0,1,0,1,1,1,0,...,1,0,0,0,0,0,0,0,1,0
4,3,4,0,0,1,0,0,1,0,1,...,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10684,3,4,0,0,0,0,0,1,0,0,...,0,0,1,1,0,0,0,0,1,0
10685,1,3,0,0,1,0,0,1,1,0,...,1,0,0,0,0,0,0,0,1,0
10686,2,4,0,0,0,0,0,0,1,0,...,0,0,1,0,1,0,0,0,1,0
10687,3,3,4,0,0,1,0,0,1,0,...,1,0,0,0,0,0,1,0,1,0


In [61]:
data_encoded = pd.concat([categorical_features, numerical],axis=1)

In [62]:
data_encoded

,coverage,education,number_of_open_complaints,region_east,region_north west,region_west region,response_yes,month_jan,employment_status_employed,employment_status_medical leave,...,vehicle_size_medsize,vehicle_size_small,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount
0,1,3,0,0,0,0,0,0,1,0,...,1,0,4809,48029,61,7,52,0,9,292
1,1,3,0,0,0,1,0,1,0,0,...,1,0,2228,92260,64,3,26,0,1,744
2,1,4,0,1,0,0,0,0,1,0,...,1,0,14947,22139,100,34,31,0,2,480
3,2,3,0,0,1,0,1,1,1,0,...,1,0,22332,49078,97,10,3,0,2,484
4,3,4,0,0,1,0,0,1,0,1,...,1,0,9025,23675,117,33,31,0,7,707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10684,3,4,0,0,0,0,0,1,0,0,...,1,0,15563,61541,253,12,40,0,7,1214
10685,1,3,0,0,1,0,0,1,1,0,...,1,0,5259,61146,65,7,68,0,6,273
10686,2,4,0,0,0,0,0,0,1,0,...,1,0,23893,39837,201,11,63,0,2,381
10687,3,3,4,0,0,1,0,0,1,0,...,1,0,11971,64195,158,0,27,4,6,618


In [63]:
data_encoded = data_encoded.apply(lambda x: x.fillna(x.mean()),axis=0)

In [64]:
data_encoded

,coverage,education,number_of_open_complaints,region_east,region_north west,region_west region,response_yes,month_jan,employment_status_employed,employment_status_medical leave,...,vehicle_size_medsize,vehicle_size_small,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount
0,1,3,0,0,0,0,0,0,1,0,...,1,0,4809,48029,61,7,52,0,9,292
1,1,3,0,0,0,1,0,1,0,0,...,1,0,2228,92260,64,3,26,0,1,744
2,1,4,0,1,0,0,0,0,1,0,...,1,0,14947,22139,100,34,31,0,2,480
3,2,3,0,0,1,0,1,1,1,0,...,1,0,22332,49078,97,10,3,0,2,484
4,3,4,0,0,1,0,0,1,0,1,...,1,0,9025,23675,117,33,31,0,7,707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10684,3,4,0,0,0,0,0,1,0,0,...,1,0,15563,61541,253,12,40,0,7,1214
10685,1,3,0,0,1,0,0,1,1,0,...,1,0,5259,61146,65,7,68,0,6,273
10686,2,4,0,0,0,0,0,0,1,0,...,1,0,23893,39837,201,11,63,0,2,381
10687,3,3,4,0,0,1,0,0,1,0,...,1,0,11971,64195,158,0,27,4,6,618


## X-Y Splitting

In [65]:
X = data_encoded.drop(['total_claim_amount'],axis = 1)
y = data_encoded['total_claim_amount']

In [66]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

X_train = pd.DataFrame(X_train, columns=X.columns)
X_test  = pd.DataFrame(X_test, columns=X.columns)

## Linear Regression

In [67]:
model=LinearRegression()
model.fit(X_train, y_train)
print(f"{model.__class__.__name__}: Train -> {model.score(X_train, y_train)}, Test -> {model.score(X_test, y_test)}")

LinearRegression: Train -> 0.7697535683402152, Test -> 0.7740652008444016


## Lasso Model

In [68]:
from sklearn.linear_model import Lasso,Ridge,ElasticNet, LinearRegression
model=Lasso(alpha=0.05)

model.fit(X_train, y_train)
print(f"{model.__class__.__name__}: Train -> {model.score(X_train, y_train)}, Test -> {model.score(X_test, y_test)}")

Lasso: Train -> 0.7697284017866969, Test -> 0.774085366769119


## Ridge Model

In [71]:
model=Ridge(alpha=1000)
model.fit(X_train, y_train)
print(f"{model.__class__.__name__}: Train -> {model.score(X_train, y_train)}, Test -> {model.score(X_test, y_test)}")

Ridge: Train -> 0.7090544278954438, Test -> 0.7111685431248354


In [86]:
def models(X,y):
    
    
    
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

    X_train = pd.DataFrame(X_train, columns=X.columns)
    X_test  = pd.DataFrame(X_test, columns=X.columns)
    
    model=LinearRegression()
    model.fit(X_train, y_train)
    print(f"{model.__class__.__name__}: Train -> {model.score(X_train, y_train)}, Test -> {model.score(X_test, y_test)}")
    
    
    model=Lasso(alpha=0.05)
    model.fit(X_train, y_train)
    print(f"{model.__class__.__name__}: Train -> {model.score(X_train, y_train)}, Test -> {model.score(X_test, y_test)}")
    
    model=Ridge(alpha=1000)
    model.fit(X_train, y_train)
    print(f"{model.__class__.__name__}: Train -> {model.score(X_train, y_train)}, Test -> {model.score(X_test, y_test)}")
    
    

            

## Feature Selection


In [76]:
lm = LinearRegression()

selector = RFE(lm, n_features_to_select= 30, step = 1, verbose = 1) # Step is how many features to add or drop everytime
selector.fit(X_train, y_train)

kept_features = selector.get_support(indices = True) #returns an array of integers corresponding to nonremoved features
kept_features = list(X_train.iloc[:,kept_features].columns)

X_train = selector.transform(X_train)
X_test  = selector.transform(X_test)

X_train = pd.DataFrame(X_train, columns=kept_features)
X_test  = pd.DataFrame(X_test, columns=kept_features)

print("Final selected features: ")
display(X_train)

Fitting estimator with 47 features.
Fitting estimator with 46 features.
Fitting estimator with 45 features.
Fitting estimator with 44 features.
Fitting estimator with 43 features.
Fitting estimator with 42 features.
Fitting estimator with 41 features.
Fitting estimator with 40 features.
Fitting estimator with 39 features.
Fitting estimator with 38 features.
Fitting estimator with 37 features.
Fitting estimator with 36 features.
Fitting estimator with 35 features.
Fitting estimator with 34 features.
Fitting estimator with 33 features.
Fitting estimator with 32 features.
Fitting estimator with 31 features.
Final selected features: 


,coverage,education,region_west region,response_yes,employment_status_employed,employment_status_medical leave,employment_status_retired,employment_status_unemployed,gender_m,location_code_suburban,...,renew_offer_type_offer3,renew_offer_type_offer4,vehicle_class_luxury car,vehicle_class_luxury suv,vehicle_class_sports car,vehicle_class_suv,vehicle_class_two-door car,vehicle_size_medsize,vehicle_size_small,monthly_premium_auto
0,1,4,1,1,0,0,0,1,1,1,...,0,0,0,0,0,0,1,1,0,65
1,3,1,1,0,1,0,0,0,1,1,...,0,0,0,0,0,0,1,0,1,106
2,1,3,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,101
3,1,4,1,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,63
4,1,4,0,0,1,0,0,0,0,1,...,0,0,0,0,0,1,0,1,0,115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8546,2,3,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,94
8547,2,4,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,98
8548,1,5,1,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,1,64
8549,1,1,0,0,1,0,0,0,1,0,...,0,0,0,0,0,1,0,1,0,106


In [80]:
X_train.columns

Index(['coverage', 'education', 'region_west region', 'response_yes',
       'employment_status_employed', 'employment_status_medical leave',
       'employment_status_retired', 'employment_status_unemployed', 'gender_m',
       'location_code_suburban', 'location_code_urban',
       'marital_status_single', 'policy_type_special auto',
       'policy_corporate l2', 'policy_personal l2', 'policy_personal l3',
       'policy_special l1', 'policy_special l2', 'policy_special l3',
       'renew_offer_type_offer2', 'renew_offer_type_offer3',
       'renew_offer_type_offer4', 'vehicle_class_luxury car',
       'vehicle_class_luxury suv', 'vehicle_class_sports car',
       'vehicle_class_suv', 'vehicle_class_two-door car',
       'vehicle_size_medsize', 'vehicle_size_small', 'monthly_premium_auto'],
      dtype='object')

In [84]:
# Running the model with 30 most important features

X = X[['coverage', 'education', 'region_west region', 'response_yes',
       'employment_status_employed', 'employment_status_medical leave',
       'employment_status_retired', 'employment_status_unemployed', 'gender_m',
       'location_code_suburban', 'location_code_urban',
       'marital_status_single', 'policy_type_special auto',
       'policy_corporate l2', 'policy_personal l2', 'policy_personal l3',
       'policy_special l1', 'policy_special l2', 'policy_special l3',
       'renew_offer_type_offer2', 'renew_offer_type_offer3',
       'renew_offer_type_offer4', 'vehicle_class_luxury car',
       'vehicle_class_luxury suv', 'vehicle_class_sports car',
       'vehicle_class_suv', 'vehicle_class_two-door car',
       'vehicle_size_medsize', 'vehicle_size_small', 'monthly_premium_auto']]

In [88]:
models(X,y)

LinearRegression: Train -> 0.7692684985492501, Test -> 0.7739714393338601
Lasso: Train -> 0.769243611154377, Test -> 0.7739727782707797
Ridge: Train -> 0.7085453997900719, Test -> 0.7120830274843468


,const,coverage,education,region_west region,response_yes,employment_status_employed,employment_status_medical leave,employment_status_retired,employment_status_unemployed,gender_m,...,renew_offer_type_offer3,renew_offer_type_offer4,vehicle_class_luxury car,vehicle_class_luxury suv,vehicle_class_sports car,vehicle_class_suv,vehicle_class_two-door car,vehicle_size_medsize,vehicle_size_small,monthly_premium_auto
0,1.0,1,3,0,0,1,0,0,0,1,...,1,0,0,0,0,0,0,1,0,61
1,1.0,1,3,1,0,0,0,0,1,0,...,0,1,0,0,0,0,0,1,0,64
2,1.0,1,4,0,0,1,0,0,0,1,...,1,0,0,0,0,1,0,1,0,100
3,1.0,2,3,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,1,0,97
4,1.0,3,4,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10684,1.0,3,4,0,0,0,0,0,1,0,...,1,0,1,0,0,0,0,1,0,253
10685,1.0,1,3,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,65
10686,1.0,2,4,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,1,0,201
10687,1.0,3,3,1,0,1,0,0,0,0,...,0,0,0,0,0,1,0,1,0,158


In [92]:


X = data_encoded.drop(['total_claim_amount'],axis = 1)

In [93]:
X_added_constant = sm.add_constant(X)
X_added_constant

,const,coverage,education,number_of_open_complaints,region_east,region_north west,region_west region,response_yes,month_jan,employment_status_employed,...,vehicle_class_two-door car,vehicle_size_medsize,vehicle_size_small,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies
0,1.0,1,3,0,0,0,0,0,0,1,...,0,1,0,4809,48029,61,7,52,0,9
1,1.0,1,3,0,0,0,1,0,1,0,...,0,1,0,2228,92260,64,3,26,0,1
2,1.0,1,4,0,1,0,0,0,0,1,...,0,1,0,14947,22139,100,34,31,0,2
3,1.0,2,3,0,0,1,0,1,1,1,...,0,1,0,22332,49078,97,10,3,0,2
4,1.0,3,4,0,0,1,0,0,1,0,...,0,1,0,9025,23675,117,33,31,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10684,1.0,3,4,0,0,0,0,0,1,0,...,0,1,0,15563,61541,253,12,40,0,7
10685,1.0,1,3,0,0,1,0,0,1,1,...,0,1,0,5259,61146,65,7,68,0,6
10686,1.0,2,4,0,0,0,0,0,0,1,...,0,1,0,23893,39837,201,11,63,0,2
10687,1.0,3,3,4,0,0,1,0,0,1,...,0,1,0,11971,64195,158,0,27,4,6


In [94]:
model = sm.OLS(y,X_added_constant).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     total_claim_amount   R-squared:                       0.771
Model:                            OLS   Adj. R-squared:                  0.770
Method:                 Least Squares   F-statistic:                     813.9
Date:                Mon, 19 Sep 2022   Prob (F-statistic):               0.00
Time:                        16:35:39   Log-Likelihood:                -67990.
No. Observations:               10689   AIC:                         1.361e+05
Df Residuals:                   10644   BIC:                         1.364e+05
Df Model:                          44                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const                            -333.9016     14.009    -23.835      0.000    -361.362    -306.441
coverage                           -8.8823      3.356     -2.647      0.008     -15.461      -2.304
education                          -4.6024      0.955     -4.820      0.000      -6.474      -2.731
number_of_open_complaints          -0.8871      0.749     -1.185      0.236      -2.355       0.580
region_east                         5.9597      5.269      1.131      0.258      -4.368      16.288
region_north west                  -1.7690      3.603     -0.491      0.623      -8.832       5.294
region_west region                  3.9904      3.439      1.160      0.246      -2.750      10.731
response_yes                      -24.6170      4.395     -5.602      0.000     -33.231     -16.003
month_jan                          -0.6280      2.728     -0.230      0.818      -5.976       4.720
employment_status_employed        -10.7233      7.054     -1.520      0.129     -24.551       3.104
employment_status_medical leave    12.7496      9.001      1.417      0.157      -4.893      30.392
employment_status_retired          -9.1083     10.554     -0.863      0.388     -29.796      11.580
employment_status_unemployed       78.6694      7.454     10.554      0.000      64.058      93.281
gender_m                           15.5878      2.745      5.679      0.000      10.207      20.968
location_code_suburban            380.1681      3.937     96.557      0.000     372.450     387.886
location_code_urban               222.1650      4.485     49.540      0.000     213.374     230.956
marital_status_married             -5.3487      4.045     -1.322      0.186     -13.278       2.580
marital_status_single              66.1488      4.645     14.242      0.000      57.045      75.253
policy_type_personal auto           0.7612      5.250      0.145      0.885      -9.529      11.052
policy_type_special auto           13.8229      7.454      1.854      0.064      -0.788      28.434
policy_corporate l2                -5.9945      8.660     -0.692      0.489     -22.969      10.980
policy_corporate l3                -0.7088      7.911     -0.090      0.929     -16.216      14.798
policy_personal l1                  4.3409      3.368      1.289      0.197      -2.261      10.943
policy_personal l2                 -1.6117      2.920     -0.552      0.581      -7.335       4.111
policy_personal l3                 -1.9680      2.648     -0.743      0.457      -7.159       3.223
policy_special l1                  38.4080     12.624      3.043      0.002      13.663      63.153
policy_special l2                 -12.3658      9.165     -1.349      0.177     -30.331       5.600
policy_special l3                 -12.2193      9.519     -1.284      0.199     -30

In [95]:
model.params[list(np.where(model.pvalues < 0.05)[0])].iloc[0:].index.tolist()
significant_features=X_added_constant[model.params[list(np.where(model.pvalues < 0.05)[0])].iloc[0:].index.tolist()]
significant_features

,const,coverage,education,response_yes,employment_status_unemployed,gender_m,location_code_suburban,location_code_urban,marital_status_single,policy_special l1,renew_offer_type_offer2,vehicle_class_sports car,vehicle_class_suv,income,monthly_premium_auto,months_since_last_claim
0,1.0,1,3,0,0,1,1,0,0,0,0,0,0,48029,61,7
1,1.0,1,3,0,1,0,1,0,1,0,0,0,0,92260,64,3
2,1.0,1,4,0,0,1,1,0,1,0,0,0,1,22139,100,34
3,1.0,2,3,1,0,1,1,0,1,0,1,0,0,49078,97,10
4,1.0,3,4,0,0,0,1,0,0,0,0,0,0,23675,117,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10684,1.0,3,4,0,1,0,1,0,0,0,0,0,0,61541,253,12
10685,1.0,1,3,0,0,0,0,1,0,0,1,0,0,61146,65,7
10686,1.0,2,4,0,0,0,0,0,0,0,0,0,0,39837,201,11
10687,1.0,3,3,0,0,0,0,1,0,0,0,0,1,64195,158,0


## Applying the model after dropping insignificant features

In [96]:
models(significant_features, y)

LinearRegression: Train -> 0.7689498755381561, Test -> 0.7737842800178454
Lasso: Train -> 0.7689429495351185, Test -> 0.7736427646214983
Ridge: Train -> 0.7068143850079668, Test -> 0.7087305585531616
